In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from PIL import Image, ImageOps, ImageDraw
import glob

In [42]:
lq_df=pd.DataFrame(
    {'left_argument':[]
    ,'right_argument' :[]
    ,'product in Z' :[]
    ,'product in Z/3':[]}
)
lq_df
for i in range(3):
    for j in range(3):
        lq_df=lq_df.append({'left_argument': i, 'right_argument': j,'product in Z':2*i-j, 'product in Z/3': (2*i-j%3) },ignore_index=True)


/var/folders/lj/f1ygs8y13w9crbf6d4d7mgcm0000gn/T/ipykernel_18638/3242063470.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lq_df=lq_df.append({'left_argument': i, 'right_argument': j,'product in Z':2*i-j, 'product in Z/3': (2*i-j%3) },ignore_index=True)
/var/folders/lj/f1ygs8y13w9crbf6d4d7mgcm0000gn/T/ipykernel_18638/3242063470.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lq_df=lq_df.append({'left_argument': i, 'right_argument': j,'product in Z':2*i-j, 'product in Z/3': (2*i-j%3) },ignore_index=True)
/var/folders/lj/f1ygs8y13w9crbf6d4d7mgcm0000gn/T/ipykernel_18638/3242063470.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lq_df=lq_df.append({'left_argument': i, 'right_argument': j,'product in Z':2*i-j, 'p

In [37]:
lq_df

,left_argument,right_argument,product in Z,product in Z/3
0,0.0,0.0,0.0,0.0
1,0.0,1.0,-1.0,-1.0
2,0.0,2.0,-2.0,-2.0
3,1.0,0.0,2.0,2.0
4,1.0,1.0,1.0,1.0
5,1.0,2.0,0.0,0.0
6,2.0,0.0,4.0,4.0
7,2.0,1.0,3.0,3.0
8,2.0,2.0,2.0,2.0


In [52]:
#grab value from entry and not return a series
lq_df.loc[lq_df['product in Z/3']==1.0]['right_argument'].values[0]

1.0

In [31]:
# Create the frames
frames = []
imgs = glob.glob("./images/*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

# Save into a GIF file that loops forever
frames[0].save('png_to_gif.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)

In [2]:
#imports the csv and zero pads the county fips
census_df=pd.read_csv('census.csv')
census_df['COUNTYFP10']=census_df['COUNTYFP10'].astype(str).str.pad(3,fillchar='0')
census_df.head()

,county_id,county,population,COUNTYFP10,latitude,longitude
0,0,Adair,7496,001,41.328528,-94.478164
1,1,Adams,3704,003,41.021656,-94.696906
2,2,Allamakee,14061,005,43.274964,-91.382751
3,3,Appanoose,12317,007,40.744683,-92.870345
4,4,Audubon,5674,009,41.679178,-94.904312


In [3]:
np.zeros(10,dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [4]:
# making a dummy dataframe to track the number of times a county was switched into another district
data={
    'county_id':[0,1,2,3,4,5,6,7,8,9],
    'district':[0,1,2,3,0,1,2,3,2,3],
    'population':[20,30,20,30,40,20,30,20,30,40],
    'num_switches':np.zeros(10,dtype=int)
}
df=pd.DataFrame(data)
df

,county_id,district,population,num_switches
0,0,0,20,0
1,1,1,30,0
2,2,2,20,0
3,3,3,30,0
4,4,0,40,0
5,5,1,20,0
6,6,2,30,0
7,7,3,20,0
8,8,2,30,0
9,9,3,40,0


In [5]:
#adds up the population in each district. This does not seem efficient as it generates the district population as a list
districts=[]
for i in range(4):
    districts.append(df['population'].loc[df['district']==i].sum())
    print(districts)

[60]
[60, 50]
[60, 50, 80]
[60, 50, 80, 90]


In [6]:
#this identifies the population of the largest county in each district
for i in range(4):
    print(    max(df['population'].loc[df['district']==i]))

40
30
30
40


In [7]:
#identifies which county in district 3 has the largest population
df['population'].loc[df['district']==3]==max(df['population'].loc[df['district']==3])

3    False
7    False
9     True
Name: population, dtype: bool

In [8]:
#check to see if the index is reset for the dataframe subset
district=df.loc[df['district']==0]
district

,county_id,district,population,num_switches
0,0,0,20,0
4,4,0,40,0


In [9]:
for i in range(4):
    district_max_pop=max(df['population'].loc[df['district']==i])
    district_ave_pop=df['population'].loc[df['district']==i].mean()
    district_min_switches=min(df['num_switches'].loc[df['district']==i])
    district=df.loc[df['district']==i] #this uses the index from the original dataframe
    # print(district)
    for j in range(district.shape[0]):
        if (district.iloc[j][2]==district_max_pop) & (district.iloc[j][3]==district_min_switches):
            print(district.iloc[j][0])
        # print(i,district.iloc[j][2])

4
1
6
8
9


County 4 comes from district 0. County 1 comes from district 1. Counties 6 and 8 come from district 2 as their populations are tied. County 9 comes from district 3.

In [12]:
#sorts the counties in the dataframe in descending order based on county population
df.sort_values('population', ascending=False)

,county_id,district,population,num_switches
4,4,0,40,0
9,9,3,40,0
1,1,1,30,0
3,3,3,30,0
6,6,2,30,0
8,8,2,30,0
0,0,0,20,0
2,2,2,20,0
5,5,1,20,0
7,7,3,20,0


In [13]:
#adding in a test column into the original dataframe
df['test_col']=np.ones(10, dtype=int)
df

,county_id,district,population,num_switches,test_col
0,0,0,20,0,1
1,1,1,30,0,1
2,2,2,20,0,1
3,3,3,30,0,1
4,4,0,40,0,1
5,5,1,20,0,1
6,6,2,30,0,1
7,7,3,20,0,1
8,8,2,30,0,1
9,9,3,40,0,1


In [14]:
#make a deep copy of the dataframe
df1=df.sort_values('population', ascending=False)
df1

,county_id,district,population,num_switches,test_col
4,4,0,40,0,1
9,9,3,40,0,1
1,1,1,30,0,1
3,3,3,30,0,1
6,6,2,30,0,1
8,8,2,30,0,1
0,0,0,20,0,1
2,2,2,20,0,1
5,5,1,20,0,1
7,7,3,20,0,1


In [16]:
#set an arbitrary limit on the population for proof of concept
pop_limit=31
#gives the column index for num_switches 
switch_index=df1.columns.get_loc('num_switches')
min_switches=min(df1['num_switches'])
for i in range(df1.shape[0]):
    if (df1.iloc[i]['num_switches']==min_switches) & (df1.iloc[i]['population']<pop_limit):
        df1.iloc[i,switch_index]+=1
        # movingcounty_index=i
        print(i)
        break
    else:
        print('This county does not satisfy the num_switch and population threshold criteria')

#this is proof of concept for identifying a county which satisfies given criteria. Add one to the num_switches value and terminate the loop once such a county is found.

This county does not satisfy the num_switch and population threshold criteria
This county does not satisfy the num_switch and population threshold criteria
2


In [25]:
pop_limit=31


In [26]:
def switch_func(dataframe, pop_limit):
    #gives the column index for num_switches 
    switch_index=dataframe.columns.get_loc('num_switches')
    min_switches=min(dataframe['num_switches'])
    for i in range(dataframe.shape[0]):
        if (dataframe.iloc[i]['num_switches']==min_switches) & (dataframe.iloc[i]['population']<pop_limit):
            dataframe.iloc[i,switch_index]+=1
            # movingcounty_index=i
            print(i)
            break
        else:
            print('This county does not satisfy the num_switch and population threshold criteria')  

In [27]:
switch_func(df1,31)

This county does not satisfy the num_switch and population threshold criteria
This county does not satisfy the num_switch and population threshold criteria
This county does not satisfy the num_switch and population threshold criteria
This county does not satisfy the num_switch and population threshold criteria
This county does not satisfy the num_switch and population threshold criteria
5


In [28]:
df1

,county_id,district,population,num_switches,test_col
4,4,0,40,0,1
9,9,3,40,0,1
1,1,1,30,1,1
3,3,3,30,1,1
6,6,2,30,1,1
8,8,2,30,1,1
0,0,0,20,0,1
2,2,2,20,0,1
5,5,1,20,0,1
7,7,3,20,0,1


In [18]:
#pulls all counties from the original dataframe whose population is less than the threshold and sorts the dataframe in decreasing order based on population
df2=df.loc[df['population']<pop_limit].sort_values('population', ascending=False)
df2

,county_id,district,population,num_switches,test_col
1,1,1,30,0,1
3,3,3,30,0,1
6,6,2,30,0,1
8,8,2,30,0,1
0,0,0,20,0,1
2,2,2,20,0,1
5,5,1,20,0,1
7,7,3,20,0,1


In [41]:
switch_index_2=df2.columns.get_loc('num_switches')
min_switches=-2
max_switches=max(df2['num_switches'])
for j in range(max_switches-min_switches+1):
    for i in range(df2.shape[0]): #number of rows in df2
        if (df2.iloc[i]['num_switches']==min_switches+j) & (min_switches+1<=max_switches):
            df2.iloc[i,switch_index_2]+=1   #adds one to num_switches
            print('threshold+',j,'and county', i,' was switched')
            break

threshold+ 2 and county 0  was switched


In [42]:
df2

,county_id,district,population,num_switches,test_col
1,1,1,30,1,1
3,3,3,30,0,1
6,6,2,30,0,1
8,8,2,30,0,1
0,0,0,20,0,1
2,2,2,20,0,1
5,5,1,20,0,1
7,7,3,20,0,1
